In [2]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
# import arabicstopwords.arabicstopwords as stp
from __future__ import unicode_literals
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from sklearn.metrics import classification_report
# from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline



In [3]:
dialect_df=pd.read_csv("final_dialect_dataset_2.csv")

In [4]:
dialect_df['dialect'].unique()

array(['IQ', 'LY', 'QA', 'PL', 'SY', 'TN', 'JO', 'MA', 'SA', 'YE', 'DZ',
       'EG', 'LB', 'KW', 'OM', 'SD', 'AE', 'BH'], dtype=object)

In [5]:
dialect_df=dialect_df.drop(columns=['Unnamed: 0'])

In [6]:
dialect_df


,dialect,content
0,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,IQ,@KanaanRema مبين من كلامه خليجي
3,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺
...,...,...
458192,BH,@Al_mhbaa_7 مبسوطين منك اللي باسطانا😅
458193,BH,@Zzainabali @P_ameerah والله ماينده ابش يختي
458194,BH,@Al_mhbaa_7 شو عملنا لك حنا تهربي مننا احنا مس...
458195,BH,@haneenalmwla الله يبارك فيها وبالعافيه 😋😋😋


In [7]:
dialect_df['content'][:100]

0      @Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1     @7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2                       @KanaanRema مبين من كلامه خليجي
3            @HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4                    @hmo2406 وين هل الغيبه  اخ محمد 🌸🌺
                            ...                        
95    @Rahhafjaseeeeem @AliAljarah16 حلوه بالعراق ان...
96    @AmeenPress سنة يسولف برفع الصبات واللي اعتبره...
97    @OmarAlmansuri هذا كله  حتى يبقى اقتصاد ايران ...
98                    @OmarAlmansuri كبسولة كل ست ساعات
99             @LaylaKilani صدقتي عزيزتي .. يسعد مساك 🌸
Name: content, Length: 100, dtype: object

In [8]:
def remove_non_ar(text):
    return re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', text)

In [9]:
dialect_df['content']=dialect_df['content'].apply(remove_non_ar)

In [10]:
stop_words =['من',
 'في',
 'على',
 'و',
 'فى',
 'يا',
 'عن',
 'مع',
 'ان',
 'هو',
 'علي',
 'ما',
 'اللي',
 'كل',
 'بعد',
 'ده',
 'اليوم',
 'أن',
 'يوم',
 'انا',
 'إلى',
 'كان',
 'ايه',
 'اللى',
 'الى',
 'دي',
 'بين',
 'انت',
 'أنا',
 'حتى',
 'لما',
 'فيه',
 'هذا',
 'واحد',
 'احنا',
 'اي',
 'كده',
 'إن',
 'او',
 'أو',
 'عليه',
 'ف',
 'دى',
 'مين',
 'الي',
 'كانت',
 'أمام',
 'زي',
 'يكون',
 'خلال',
 'ع',
 'كنت',
 'هي',
 'فيها',
 'عند',
 'التي',
 'الذي',
 'قال',
 'هذه',
 'قد',
 'انه',
 'ريتويت',
 'بعض',
 'أول',
 'ايه',
 'الان',
 'أي',
 'منذ',
 'عليها',
 'له',
 'ال',
 'تم',
 'ب',
 'دة',
 'عليك',
 'اى',
 'كلها',
 'اللتى',
 'هى',
 'دا',
 'انك',
 'وهو',
 'ومن',
 'منك',
 'نحن',
 'زى',
 'أنت',
 'انهم',
 'معانا',
 'حتي',
 'وانا',
 'عنه',
 'إلي',
 'ونحن',
 'وانت',
 'منكم',
 'وان',
 'معاهم',
 'معايا',
 'وأنا',
 'عنها',
 'إنه',
 'اني',
 'معك',
 'اننا',
 'فيهم',
 'د',
 'انتا',
 'عنك',
 'وهى',
 'معا',
 'آن',
 'انتي',
 'وأنت',
 'وإن',
 'ومع',
 'وعن',
 'معاكم',
 'معاكو',
 'معاها',
 'وعليه',
 'وانتم',
 'وانتي',
 '¿',
 '|']

In [11]:
def removing_ar_stopwords(text):
    """
        Here we remove all Arabic stop words
        
    """
      # if read it from file
#     ar_stopwords_list = open("arabic_stopwords.txt", "r") 
#     stop_words = ar_stopwords_list.read().split("\n")
#     stop_words = []
    original_words = []
    words = word_tokenize(text) # it works on one hadith not list
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence


In [12]:
dialect_df['content']=dialect_df['content'].apply(removing_ar_stopwords)

In [13]:
def remove_duplicate_letters(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [14]:
dialect_df['content']=dialect_df['content'].apply(remove_duplicate_letters)

In [15]:
dialect_df.to_csv('final_data_cleaned.csv')